# SVM - Climate Sentiment Multiclass Classification
## CS522 Project

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC,SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from Common.DataCenter import data_center
from Common.LSI import SKLearnLSA
from Common.UtilFuncs import DataSize
%matplotlib inline
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
    
TrainSizeBaseLine = DataSize.GetTrainSizeBaseline()
TrainSizeWithNoisyData = DataSize.GetTrainSizeWithNoisyData()
TestDataSize = DataSize.GetTestDataSize()
NoiseDataSize = DataSize.GetNoiseDataSize()
ValidationDataSize = DataSize.GetValidationDataSize()

time: 155 µs (started: 2022-04-09 10:40:09 +08:00)


### Text preprocessing

In [2]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    # Convert texts to vectors
    lsa = SKLearnLSA()
    lsa.BuildModel(X_train + X_test, 800)
    X_train_vec = lsa.Query2LatentSpace(X_train)
    X_test_vec = lsa.Query2LatentSpace(X_test)
    return X_train_vec, X_test_vec


time: 238 µs (started: 2022-04-09 10:40:09 +08:00)


### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


time: 206 µs (started: 2022-04-09 10:40:09 +08:00)


### Run SVM and evaluate the results

In [4]:
# parameter:  vectorised X and encoded y of training set and test set
def evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    #SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight="balanced"), n_jobs=-1)
    SVM = OneVsRestClassifier(SVC(gamma='auto', class_weight="balanced"), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    prediction      = SVM.predict(X_test_vec)

    # Evaluate the results
    macro_f1        = f1_score(y_test_vec, prediction, average='macro')
    weighted_f1     = f1_score(y_test_vec, prediction, average='weighted')
    macro_precision = precision_score(y_test_vec, prediction, average='macro')
    macro_recall    = recall_score(y_test_vec, prediction, average='macro')

    return macro_f1, weighted_f1, macro_precision, macro_recall


time: 374 µs (started: 2022-04-09 10:40:09 +08:00)


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # Run SVM and evaluate the results
    macro_f1, weighted_f1, macro_precision, macro_recall = \
        evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Show the indicators
    print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
          (macro_f1, weighted_f1, macro_precision, macro_recall))
    return X_train_vec


time: 317 µs (started: 2022-04-09 10:40:09 +08:00)


### Main entry

**Load the database and split it into training set, test set, noisy set, validation set**

In [6]:
dc = data_center("twitter_sentiment_data_clean.csv", test_size=TestDataSize, noisy_size=NoiseDataSize, validation_size=ValidationDataSize)

print("####################################################")
print("Total data size: ",       dc.get_len())
print("Total train data size: ", dc.get_train_len())
print("Total test data size: ",  dc.get_test_len())

####################################################
Total data size:  40908
Total train data size:  32908
Total test data size:  4000
time: 81.8 ms (started: 2022-04-09 10:40:09 +08:00)


**Get the test set for evaluation**

In [7]:
# distribution of training set
train_distribution = None


time: 141 µs (started: 2022-04-09 10:40:09 +08:00)


**Prepare the noisy set.**

In [8]:
# distribution of external noisy
external_noisy_distribution = [0.25, 0.25, 0.25, 0.25]
lstNoisyInfo = [("mislabeled",dc.get_noisy_len())]
print("Noisy set size is %d"                % dc.get_noisy_len())

# add the external noisy data (irrelevant texts)
added_size = dc.add_noisy(noisy_source="irrelevant",
                          distribution = external_noisy_distribution, size = 6000) # max size: 34259
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("irrelevant",added_size))

# add the external noisy data (translated texts)
added_size = dc.add_noisy(noisy_source="translated",
                          distribution = "reserve_labels", size = 6000) # max size: 6146
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("translated",added_size))

print("Noisy set new size is %d"                % dc.get_noisy_len())



Noisy set size is 3000
./Noisy/irrelevant/Twitter and Reddit Sentimental analysis Dataset.csv
./Noisy/irrelevant/Liberals vs Conservatives on Reddit.csv
./Noisy/irrelevant/Twitter Sentiment Analysis.csv
./Noisy/irrelevant/Portuguese Tweets for Sentiment Analysis(外文).csv
./Noisy/irrelevant/Tweets with Sarcasm and Irony.csv
./Noisy/irrelevant/Disaster Tweets.csv
./Noisy/irrelevant/Odia News Dataset(foreign).csv
./Noisy/irrelevant/CNN_Articels_clean.csv
./Noisy/irrelevant/IMDB Movie Ratings Sentiment Analysis.csv
./Noisy/irrelevant/fake_new.csv
./Noisy/irrelevant/Amazon Product Reviews.csv
./Noisy/irrelevant/covid19-tweets.csv
Cannot generate the set with size 3000 and distribution None!


NameError: name 'exit' is not defined

time: 545 ms (started: 2022-04-09 10:40:09 +08:00)


In [ ]:
X_test, y_test = dc.get_test()

**Run experiments with different training sets, and use the same test set.**

In [ ]:
print("-----------------------------------------------")
for size in TrainSizeBaseLine:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size, train_distribution)
    print("Training set size: %d samples (%.1f%%): " % (len(X_train), len(y_train)/dc.get_train_len()*100))

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

print("-----------------------------------------------")
xtrainvec = None
for size in TrainSizeWithNoisyData:
    # Get a noisy training set
    X_train, y_train = dc.get_train_with_noisy(size[0], size[1], train_distribution)
    print("Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))

    # Do an experiment
    xtrainvec = do_experiment(X_train, y_train, X_test, y_test)

In [ ]:
X_train, y_train = dc.get_train(2000)
# Convert texts to vectors
X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

# Run SVM and evaluate the results
macro_f1, weighted_f1, macro_precision, macro_recall = \
    evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

# Show the indicators
print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
        (macro_f1, weighted_f1, macro_precision, macro_recall))